This program involves the use to a database in Python using SQLite3. We will be importing a couple of files and libraries as well. 

In [1]:
#importing the necessary libraries and modules
import sqlite3
import pandas as pd

In [2]:
#using the connect function from the sqlite module to connect to the database and obtain a connection object
connection = sqlite3.connect('books.db')

Using pandas to display the database's contents

In [3]:
pd.options.display.max_columns = 10

In [4]:
pd.read_sql('SELECT * FROM authors', connection, index_col = ['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [5]:
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


Pulling the ISBN from the database

In [6]:
pd.read_sql('SELECT * FROM author_ISBN', connection)

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673
5,1,0132151006
6,2,0132151006
7,3,0132151006
8,1,0134743350
9,2,0134743350


Using the 'SELECT' query in order to pull data from the database

In [7]:
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


Using the 'WHERE' clause to pull data from the database

In [8]:
#pulling the data to display the title and edition if the book has a copyright is older than 2016
pd.read_sql("""SELECT title, edition, copyright FROM titles WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


Pattern matching. This pulls data that have some kind of similar attribute

In [9]:
#Pulling and sorting the data that all have the same last name.
pd.read_sql("""SELECT id, first, last FROM authors WHERE first LIKE '_b%'""", connection, index_col = ['id'])

,first,last
id,,
3,Abbey,Deitel


Order by clause. This displays the data in ascending order

In [10]:
#Pulling the titles within the database and displaying them in ascending order
pd.read_sql('SELECT title FROM titles ORDER BY title ASC', connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


Sorting by multiple columns. This displays the data and sorts it based on what we want to sort it by

In [11]:
#Pulling the first and last name of the authors from the database. but just displaying the rows 'first' and 'last' indicating the first and last names
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last, first""", connection, index_col = ['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [12]:
#Pulling the first and last name of the authors from the database. 
#But just displaying the rows 'first' and 'last' indicating the first and last names and sorting them in decending order
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last DESC, first ASC""", connection, index_col = (['id']))

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


Combining WHERE and ORDER BY clauses. This all can be combined into one query rather than running two differet queries.

In [13]:
#This pulls data from the ISBN, Title, Edition, and copyright of eachbook and displays the ones whose title ends with 'How to Program'
pd.read_sql("""SELECT isbn, title, edition, copyright FROM titles WHERE title LIKE '%How To Program' ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


Merging Data from Multiple Tables: INNER JOIN. This merges data from multiple database tables.

In [14]:
#This pulls data from the ISBN table and merges them with the authors table
#Since we have multiple authors with multiple ISBN's the database will display one row per ISBN regarding if its the same author or not
pd.read_sql("""SELECT first, last, isbn FROM authors INNER JOIN author_isbn ON authors.id = author_ISBN.id ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


INSERT INTO statement. This inserts a row into a table

In [15]:
cursor = connection.cursor()

In [16]:
#Inserting a new vakue into the authors table
cursor = cursor.execute("""INSERT INTO authors (first, last) VALUES ('Sue', 'Red')""")

In [17]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col = ['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


UPDATE STATEMENT. This statement updates or modifies an existing value from the database.

In [18]:
#updating the previously added record and changing/updating the last name into another value
cursor = cursor.execute("""UPDATE authors SET last = 'Black' WHERE last = 'Red' AND first = 'Sue'""")

In [19]:
#displays how many rows we changed/added
cursor.rowcount

1

In [20]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col = ['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


DELETE STATEMENT. This statement deletes/removes rows from a table

In [21]:
#Deleting the previously added record. Rather than deleting it by finding the first name and the last name we specify the ID that the value has in the table and just calling that id
cursor = cursor.execute('DELETE FROM authors WHERE id = 6')

In [22]:
#displaying how many rows were edited
cursor.rowcount

1

In [23]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col= ['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [24]:
#stops and closes the database
connection.close()